In [ ]:
#Este script requiere de TensorFlow 1 y Keras
%tensorflow_version 1.x
import os
import tensorflow as tf
from keras import backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
'''
%cd '/content/drive/My Drive/bin/'
!ls
'''

[Errno 2] No such file or directory: '/content/drive/My Drive/bin/'
/content/drive/My Drive/bin
cnn4_train_2.h5  cnn4_train_2.json  h5_to_pb.ipynb


In [ ]:
'''
	Nombre:     freeze_keras_model (Keras y TensorFlow 1)
	Funcion:    Convierte modelos de Keras en frozen graphs de TF1 y lo guarda
	Entradas:   Path de los pesos H5 del modelo en Keras
				      Path del json del modelo en Keras
	Salidas:    Ninguno, pero genera y guarda un Frozen graph de TF1
'''
def freeze_keras_model(path_h5_weights, path_json, keep_var_names=None, clear_devices=True):
  K.set_learning_phase(0)
  
  reader = open(path_json, "r")
  json_data=reader.read()
  clasificador=tf.keras.models.model_from_json(json_data)
  clasificador.load_weights(path_h5_weights)

  session = K.get_session()
  graph = session.graph
  output_names = [out.op.name for out in clasificador.outputs]

  with graph.as_default():
      freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
      output_names = output_names or []
      output_names += [v.op.name for v in tf.global_variables()]
      input_graph_def = graph.as_graph_def()
      if clear_devices:
          for node in input_graph_def.node:
              node.device = ""
      frozen_graph = tf.graph_util.convert_variables_to_constants(
          session, input_graph_def, output_names, freeze_var_names)
  
  tf.io.write_graph(graph_or_graph_def=frozen_graph,
					logdir="./",
					name="frozen_graph.pb",
					as_text=False)
  print("Modelo convertido a TF1 y guardado con nombre: frozen_graph.pb")


freeze_keras_model('cnn4_train_2.h5', 'cnn4_train_2.json')

INFO:tensorflow:Froze 88 variables.
INFO:tensorflow:Converted 88 variables to const ops.


In [ ]:
!ls

cnn4_train_2.h5  cnn4_train_2.json  frozen_graph.pb  h5_to_pb.ipynb
